In [ ]:
from google.colab import files

In [ ]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d sobhanmoosavi/us-accidents

Dataset URL: https://www.kaggle.com/datasets/sobhanmoosavi/us-accidents
License(s): CC-BY-NC-SA-4.0
 96% 629M/653M [00:06<00:00, 44.7MB/s]
100% 653M/653M [00:06<00:00, 101MB/s] 


In [ ]:
!unzip us-accidents.zip

Archive:  us-accidents.zip
  inflating: US_Accidents_March23.csv  


In [ ]:
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!python rapidsai-csp-utils/colab/pip-install.py

Cloning into 'rapidsai-csp-utils'...
remote: Enumerating objects: 597, done.
remote: Counting objects: 100% (163/163), done.
remote: Compressing objects: 100% (81/81), done.
remote: Total 597 (delta 128), reused 82 (delta 82), pack-reused 434 (from 3)
Receiving objects: 100% (597/597), 196.59 KiB | 15.12 MiB/s, done.
Resolving deltas: 100% (302/302), done.
Installing RAPIDS remaining 25.04 libraries
Using Python 3.11.13 environment at: /usr
Resolved 175 packages in 12.10s
 Downloaded cuproj-cu12
 Downloaded pylibcugraph-cu12
 Downloaded shapely
 Downloaded cuspatial-cu12
 Downloaded cucim-cu12
 Downloaded dask
 Downloaded datashader
 Downloaded bokeh
 Downloaded rmm-cu12
 Downloaded libkvikio-cu12
 Downloaded cudf-cu12
 Downloaded ucx-py-cu12
 Downloaded cugraph-cu12
 Downloaded librmm-cu12
 Downloaded libcuspatial-cu12
 Downloaded cuml-cu12
 Downloaded libraft-cu12
 Downloaded pylibcudf-cu12
 Downloaded nvidia-nvcomp-cu12
 Downloaded raft-dask-cu12
 Downloaded libcuml-cu12
 Downloaded

In [ ]:
import cudf    # pandas
import cupy    # numpy

In [ ]:
df = cudf.read_csv('/content/US_Accidents_March23.csv')
df.info()

<class 'cudf.core.dataframe.DataFrame'>
RangeIndex: 7728394 entries, 0 to 7728393
Data columns (total 46 columns):
 #   Column                 Dtype
---  ------                 -----
 0   ID                     object
 1   Source                 object
 2   Severity               int64
 3   Start_Time             object
 4   End_Time               object
 5   Start_Lat              float64
 6   Start_Lng              float64
 7   End_Lat                float64
 8   End_Lng                float64
 9   Distance(mi)           float64
 10  Description            object
 11  Street                 object
 12  City                   object
 13  County                 object
 14  State                  object
 15  Zipcode                object
 16  Country                object
 17  Timezone               object
 18  Airport_Code           object
 19  Weather_Timestamp      object
 20  Temperature(F)         float64
 21  Wind_Chill(F)          float64
 22  Humidity(%)            float64
 23  

In [ ]:
# dropping useless columns
cols_to_drop = ['ID', 'Airport_Code', 'Zipcode', 'County']
df.drop(columns=cols_to_drop, inplace=True)

df.info()

<class 'cudf.core.dataframe.DataFrame'>
RangeIndex: 7728394 entries, 0 to 7728393
Data columns (total 42 columns):
 #   Column                 Dtype
---  ------                 -----
 0   Source                 object
 1   Severity               int64
 2   Start_Time             object
 3   End_Time               object
 4   Start_Lat              float64
 5   Start_Lng              float64
 6   End_Lat                float64
 7   End_Lng                float64
 8   Distance(mi)           float64
 9   Description            object
 10  Street                 object
 11  City                   object
 12  State                  object
 13  Country                object
 14  Timezone               object
 15  Weather_Timestamp      object
 16  Temperature(F)         float64
 17  Wind_Chill(F)          float64
 18  Humidity(%)            float64
 19  Pressure(in)           float64
 20  Visibility(mi)         float64
 21  Wind_Direction         object
 22  Wind_Speed(mph)        float64
 2

In [ ]:
print(cupy.unique(df.Severity))

df['Severity'] = df['Severity'].astype('uint8')
df.info()

[1 2 3 4]
<class 'cudf.core.dataframe.DataFrame'>
RangeIndex: 7728394 entries, 0 to 7728393
Data columns (total 42 columns):
 #   Column                 Dtype
---  ------                 -----
 0   Source                 object
 1   Severity               uint8
 2   Start_Time             object
 3   End_Time               object
 4   Start_Lat              float64
 5   Start_Lng              float64
 6   End_Lat                float64
 7   End_Lng                float64
 8   Distance(mi)           float64
 9   Description            object
 10  Street                 object
 11  City                   object
 12  State                  object
 13  Country                object
 14  Timezone               object
 15  Weather_Timestamp      object
 16  Temperature(F)         float64
 17  Wind_Chill(F)          float64
 18  Humidity(%)            float64
 19  Pressure(in)           float64
 20  Visibility(mi)         float64
 21  Wind_Direction         object
 22  Wind_Speed(mph)        

In [ ]:
float_cols = df.select_dtypes(include='float64').columns
df[float_cols] = df[float_cols].astype('float32')

df.info()

<class 'cudf.core.dataframe.DataFrame'>
RangeIndex: 7728394 entries, 0 to 7728393
Data columns (total 42 columns):
 #   Column                 Dtype
---  ------                 -----
 0   Source                 object
 1   Severity               uint8
 2   Start_Time             object
 3   End_Time               object
 4   Start_Lat              float32
 5   Start_Lng              float32
 6   End_Lat                float32
 7   End_Lng                float32
 8   Distance(mi)           float32
 9   Description            object
 10  Street                 object
 11  City                   object
 12  State                  object
 13  Country                object
 14  Timezone               object
 15  Weather_Timestamp      object
 16  Temperature(F)         float32
 17  Wind_Chill(F)          float32
 18  Humidity(%)            float32
 19  Pressure(in)           float32
 20  Visibility(mi)         float32
 21  Wind_Direction         object
 22  Wind_Speed(mph)        float32
 2

In [ ]:
category_cols = ['Source', 'City', 'State', 'Timezone',
                 'Wind_Direction', 'Weather_Condition', 'Sunrise_Sunset',
                 'Civil_Twilight', 'Nautical_Twilight', 'Astronomical_Twilight']

for col in category_cols:
  df[col] = df[col].astype('category')


df.info()

<class 'cudf.core.dataframe.DataFrame'>
RangeIndex: 7728394 entries, 0 to 7728393
Data columns (total 42 columns):
 #   Column                 Dtype
---  ------                 -----
 0   Source                 category
 1   Severity               uint8
 2   Start_Time             object
 3   End_Time               object
 4   Start_Lat              float32
 5   Start_Lng              float32
 6   End_Lat                float32
 7   End_Lng                float32
 8   Distance(mi)           float32
 9   Description            object
 10  Street                 object
 11  City                   category
 12  State                  category
 13  Country                object
 14  Timezone               category
 15  Weather_Timestamp      object
 16  Temperature(F)         float32
 17  Wind_Chill(F)          float32
 18  Humidity(%)            float32
 19  Pressure(in)           float32
 20  Visibility(mi)         float32
 21  Wind_Direction         category
 22  Wind_Speed(mph)        